#### Load packages

In [1]:
import pandas as pd
import numpy as np
import itertools
from pulp import *

from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


import warnings
warnings.filterwarnings("ignore")

#### Load data

In [2]:
target_hours_per_week = pd.read_excel("OR Scheduling.xlsx", sheet_name="Target Hours per Week")
target_hours_per_week.set_index(["Department"], inplace=True)


teams_available_per_week = pd.read_excel("OR Scheduling.xlsx", sheet_name="Available teams per week")
teams_available_per_week.set_index(["Department"], inplace=True)


max_daily_OR_requirement = pd.read_excel("OR Scheduling.xlsx", sheet_name="Max Daily OR requirements")
max_daily_OR_requirement.set_index(["Department"], inplace=True)


weekly_OR_requirement = pd.read_excel("OR Scheduling.xlsx", sheet_name="Weekly OR requirements")
weekly_OR_requirement.set_index(["Department"], inplace=True)

In [3]:
display_side_by_side(
    dfs=[target_hours_per_week, teams_available_per_week, max_daily_OR_requirement, weekly_OR_requirement], 
    captions=["Target Hours Per Week",
              "Teams Available Per Week", 
              "Maximum Daily OR requirement",
              "Weekly_OR_requirement"])

#### Decision variables

In [4]:
teams = teams_available_per_week.index.to_list()
day_of_week = teams_available_per_week.columns.to_list()

var_dict = LpVariable.dicts("Allotment",
                            [(dept, day) for dept in teams for day in day_of_week], 
                            lowBound = 0, 
                            cat = "Integer")

#### Model initialization

In [5]:
model = LpProblem("Optimized Allocation of Hospital Operating Rooms", LpMaximize)

#### Objective function

In [6]:
model += lpSum([8*var_dict[(team,days)]/target_hours_per_week.loc[team, "Target_Hours"]\
                for team in teams for days in day_of_week])

#### Constraints

In [7]:
#1 At most 10 ORs can be assigned per day
for day in day_of_week:
    model += lpSum([var_dict[(team, day)] for team in teams]) <= 10



#2 The number of ORs allocaed to a department on that day cannot exceed the number of surgery teams available that day
for team in teams:
    for day in day_of_week:
        model += var_dict[(team, day)] <= teams_available_per_week.loc[team, day]

        
    
#3 Meet daily minimums and maximums of each department
for team in teams:
    for day in day_of_week:
        model += var_dict[(team, day)] <= max_daily_OR_requirement.loc[team, day]

        
        
#4 Meet weekly minimums and maximums of each department
for team in teams:
    model += lpSum([var_dict[(team, day)] for day in day_of_week]) >= weekly_OR_requirement.loc[team, "Minimum"]
    model += lpSum([var_dict[(team, day)] for day in day_of_week]) <= weekly_OR_requirement.loc[team, "Maximum"]

#### Optimal Solution

In [8]:
model.solve()

if LpStatus[model.status]=="Optimal":
    optimal_soln = pd.DataFrame([(v.name, int(v.varValue)) for v in model.variables() if v.varValue!=0],
                                columns=["Decisions", "# ORs Allocated"])

#### Display results

In [9]:
display_side_by_side(dfs = [optimal_soln[optimal_soln["Decisions"].str.contains(d)].reset_index(drop=True)\
                            for d in day_of_week],
                     captions = ["Operating Room Schedule for "+i for i in day_of_week])

,Decisions,# ORs Allocated
0,"Allotment_('Cardiology',_'Monday')",3
1,"Allotment_('General_Surgery',_'Monday')",5
2,"Allotment_('Neurosurgery',_'Monday')",1
3,"Allotment_('Orthopaedics',_'Monday')",1
,Decisions,# ORs Allocated
0,"Allotment_('Cardiology',_'Tuesday')",3
1,"Allotment_('General_Surgery',_'Tuesday')",3
2,"Allotment_('Gyneacology',_'Tuesday')",2
3,"Allotment_('Neurosurgery',_'Tuesday')",1
4,"Allotment_('Orthopaedics',_'Tuesday')",1
